In [ ]:
# Title: Fig3----
# A régler: Si on prend 1978 pas tous les fichiers dans une année donc pb
# Arthur BARREAU: 06 Février 2025

In [ ]:
system("conda install -y r-sf=0.9_8")

In [4]:
system("conda install -y r-terra=1.2_10")

In [5]:
system("conda install -y r-dplyr")

In [6]:
system("conda install -y r-stringr")

In [7]:
system("conda install -y r-RColorBrewer")

In [ ]:
library(terra)
library(dplyr)
library(stringr)
library(RColorBrewer)
library(sf)
library(jsonlite) 
# 2 VARIABLE ----
months_days <- list(
  Jan = 31, Feb = 28, Mar = 31, Apr = 30, May = 31, Jun = 30, 
  Jul = 31, Aug = 31, Sep = 30, Oct = 31, Nov = 30, Dec = 31
)

# 3 FUNCTION ----

# Create directories for storing TIFF and Shapefile data
create_directories <- function(year, month) {
  tif_path <- sprintf("%s_%s_tif_folder", year,month)
  zip_path <- sprintf("%s_%s_zip_folder", year,month)
  shp_path <- sprintf("%s_%s_shapefile_folder", year,month)
  tif_mean_path <- sprintf("%s_%s_folder_tif_1981_2010", year,month)
  dir.create(tif_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(zip_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(shp_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(tif_mean_path, showWarnings = FALSE, recursive = TRUE)
  return(list(tif_path = tif_path, zip_path = zip_path, shp_path = shp_path, tif_mean_path=tif_mean_path))
}

# Download daily TIFF files
download_tiff_files <- function(path, year, start_month, month_index) {
  month_index <- match(start_month, names(months_days))  
  month_folder <- sprintf("%02d_%s", month_index, start_month) 
  
  for (day in 1:months_days[[start_month]]) {
    date_str <- sprintf("%s%02d%02d", year, month_index, day)  
    url <- paste0("https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/", year, "/", month_folder, "/S_", date_str, "_concentration_v3.0.tif")
    destfile <- file.path(path, sprintf("S_%s_concentration.tif", date_str))
    tryCatch({
      download.file(url, destfile)
    },error=function(e){})
    
  }
}


# Compute the monthly mean raster
compute_monthly_mean <- function(path, start_year, month_index) {
  tif_files <- list.files(path, pattern = "\\.tif$", full.names = TRUE)
  rasters <- rast(tif_files)
  rasters[rasters == 2550] <- NA  # Remplacer les valeurs 2550 par NA
  mean_raster <- mean(rasters, na.rm = TRUE)  
  return(mean_raster)
}


# Download and process shapefiles
download_shapefiles <- function(zip_path, shp_path, month_index) {
  month_index <- sprintf("%02d", month_index) 
  url <- paste0("https://noaadata.apps.nsidc.org/NOAA/G02135/south/monthly/shapefiles/shp_median/median_extent_S_",
                month_index, "_1981-2010_polyline_v3.0.zip")
  destfile <- file.path(zip_path, sprintf("median_extent_S_%s.zip", month_index))
  tryCatch({
    download.file(url, destfile)
    unzip(destfile, exdir = shp_path) 
  },error=function(e){})
}

# Merge shapefiles and compute mean polygon
process_shapefiles <- function(shp_path) {
  shp_files <- list.files(shp_path, pattern = "\\.shp$", full.names = TRUE)
  polygons <- lapply(shp_files, st_read)
  return(polygons)
}

download_tiff_1981_2010 <- function(path, start_year, end_year, month_index) {
  month_index <- sprintf("%02d", month_index) 
  base_url <- "https://noaadata.apps.nsidc.org/NOAA/G02135/south/monthly/geotiff/"
  
  months <- c("01_Jan", "02_Feb", "03_Mar", "04_Apr", "05_May", "06_Jun",
              "07_Jul", "08_Aug", "09_Sep", "10_Oct", "11_Nov", "12_Dec")
  month_folder <- months[as.integer(month_index)]
  
  for (year in start_year:end_year) {
    date_str <- sprintf("S_%s%s_concentration_v3.0.tif", year, month_index)
    url <- paste0(base_url, month_folder, "/", date_str)
    destfile <- file.path(path, date_str)
    tryCatch({
      download.file(url, destfile, mode = "wb")
    },error=function(e){})
  }
}


compute_mean_raster_1981_2010 <- function(path,month_index) { 
  month_index <- sprintf("%02d", month_index) 
  files <- list.files(path, pattern = "\\.tif$", full.names = TRUE)
  rasters <- rast(files)
  mean_raster_1981_2010 <- mean(rasters, na.rm = TRUE)  
  return(mean_raster_1981_2010)
}

# Create color palettes and breaks for visualization
create_colors_and_breaks <- function() {
  breaks <- c(0,1, seq(150, 1000, by = 50),1050, 2531, 2541)
  
  colors_glacier <- rev(c("white",
                          "#F7FCFF", "#E4F4FE", "#D0ECFE", "#BCE4FE", "#A8DCFD",
                          "#94D5FD", "#7DCCFD", "#6AC4FC", "#57BCFC", "#45B4FC",
                          "#31ABFC", "#23A3FC", "#1A9BFC", "#1994F9", "#178CF2",
                          "#1684EB", "#137AE3", "#093C70" ))
  
  colors <- c(
    rgb(9, 60, 112, maxColorValue = 255),
    colors_glacier,  # Dégradé bleu glacier (1 à 1000)
    rgb(0, 0, 0, maxColorValue = 255),      # Ligne côtière noire (2530)
    rgb(119, 119, 119, maxColorValue = 255) # Terre grise (2540)
  )
  return(list(colors = colors, breaks = breaks))
}

# Visualization and saving results
save_plot <- function(raster, colors, breaks, polygons, year, month,area_list) {
  plot(raster, col = colors, breaks = breaks, legend = FALSE, axes = FALSE, box = FALSE,mar = c(0, 0, 0, 0))
  plot(st_geometry(polygons[[1]]), add = TRUE, pch = 21, col = 'red', cex = 1.5)
  plot(st_geometry(area_list), add = TRUE, lwd = 1.5, border = 'red',mar=c(0,0,0,0))
  
  zones <- st_read("asd/asd-shapefile-EPSG6932.shp")    
  centroids <- st_centroid(st_geometry(zones))
  text(st_coordinates(centroids), labels = zones$GAR_Long_L, col = "red", cex = 0.8) 
  #add_labels(mode = 'auto', layer = 'ASDs', fontsize =1, col = 'red')
  
}

# ANOMALY ---------------

filter_raster_values <- function(raster) {
  raster[] <- ifelse(raster[] >= 1 & raster[] <= 150, 0, raster[])
  return(raster)
}

compute_anomaly <- function(mean_raster, mean_1981_2010) {
  mean_raster <- filter_raster_values(mean_raster)
  mean_1981_2010 <- filter_raster_values(mean_1981_2010)
  
  anomaly <- mean_raster - mean_1981_2010
  
  minmax_value <- minmax(anomaly)
  min_anomaly <- minmax_value[1]
  max_anomaly <- minmax_value[2]
  
  breaks_neg <- seq(min_anomaly/2-150, -1, length.out = 11)  
  breaks_pos <- seq(1, max_anomaly/2+150, length.out = 11) 
  breaks <- c(min_anomaly,breaks_neg, breaks_pos,max_anomaly)
  
  col_fun <- c("#053061", "#114781", "#1D5FA2", "#2B73B3", "#3A87BD", "#4F9BC7", "#71B0D3", 
               "#93C6DE", "#B1D5E7", "#CCE2EF", "#DEEBF2", "white", "#F8F1ED", "#FBE5D8", "#FCD7C2", 
               "#F8BFA4", "#F4A683", "#E8896C", "#DB6B55", "#CC4C44", "#BD2D35", "#A81529", 
               "#870A24")
  
  return(list(anomaly = anomaly, breaks = breaks, col_fun = col_fun))
}


save_anomaly_plot <- function(anomaly_data,mean_raster, year, month,area_list) {
  plot(anomaly_data$anomaly, 
       col = anomaly_data$col_fun, 
       breaks = anomaly_data$breaks, legend = FALSE, axes = FALSE, box = TRUE, mar = c(0, 0, 0, 0))
  plot(mean_raster, col = c(rgb(0, 0, 0, maxColorValue = 255),rgb(119, 119, 119, maxColorValue = 255)), breaks = c(2529,2531,2540),add=TRUE, legend = FALSE, axes = FALSE, box = FALSE)
  plot(st_geometry(area_list), add = TRUE, lwd = 1.5, border = 'red',mar=c(0,0,0,0))
 
  zones <- st_read("asd/asd-shapefile-EPSG6932.shp")    
  centroids <- st_centroid(st_geometry(zones))
  text(st_coordinates(centroids), labels = zones$GAR_Long_L, col = "red", cex = 0.8)  # Ajouter les labels
  
  
  #add_labels(mode = 'auto', layer = 'ASDs', fontsize =1, col = 'red')
  
}

mask_subarea=function(area_list,raster){
  output_dir <- "asd"
  dir.create(output_dir, showWarnings = FALSE)
  urls_asd <- c(
    "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG6932.shp",
    "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG6932.shx",
    "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG6932.dbf",
    "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG6932.prj",
    "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/CCAMLR_ASD_EPSG6932.cst"
  )
  
  # Define destination file paths
  destfiles <- file.path(output_dir, c(
    "asd-shapefile-EPSG6932.shp",
    "asd-shapefile-EPSG6932.shx",
    "asd-shapefile-EPSG6932.dbf",
    "asd-shapefile-EPSG6932.prj",
    "asd-shapefile-EPSG6932.cst"
  ))
  
  # Download each file and save to the output directory
  for (i in seq_along(urls)) {
    download.file(urls[i], destfiles[i], mode = "wb")
  }
  
  ASD <- st_read(file.path(output_dir, "asd-shapefile-EPSG6932.shp"))
  
  subareas_selected <- ASD %>% filter(GAR_Short_ %in% area_list )
  bbox <- st_bbox(subareas_selected)
  bbox_polygon <- st_as_sfc(bbox)
  bbox_vect <- vect(bbox_polygon)
  mean_raster_crop <- crop(raster, bbox_vect)  # Ajuste l'étendue
  mean_raster_mask <- mask(mean_raster_crop, bbox_vect)  # Masque l'extérieur
  return(list(mean_raster=mean_raster_mask, subareas_selected=subareas_selected))
}


# 4 MAIN ----

# Lire les paramètres depuis le fichier JSON
json_path <- "galaxy_inputs/galaxy_inputs.json"
json_data <- fromJSON(json_path)

start_month <- json_data$month
start_year <- json_data$year
chose <- json_data$ConcentrationXandXorXAnomalies
subarea <- json_data$subarea
month_list=resultat <- unlist(strsplit(start_month, "[,]"))
year_list=resultat <- unlist(strsplit(start_year, "[,]"))
chose_list=resultat <- unlist(strsplit(chose, "[_]"))
area_list=resultat <- unlist(strsplit(subarea, "[,]"))
count = 0



for(i in 1:length(month_list)){
  print(i)
  i = 1
  paths <- create_directories(year_list[i], month_list[i])
  month_index <- match(month_list[i], names(months_days))
  
  download_tiff_files(paths$tif_path, year_list[i] ,month_list[i], month_index)
  mean_raster <- compute_monthly_mean(paths$tif_path, year_list[i], month_index)
  
  mask_return <- mask_subarea(area_list,mean_raster)
  subareas_selected <- mask_return$subareas_selected
  mean_raster_mask <- mask_return$mean_raster
  
  
  download_shapefiles(paths$zip_path, paths$shp_path, month_index)
  polygons <- process_shapefiles(paths$shp_path)
  
  color_breaks <- create_colors_and_breaks()
  
  if(count == 0){
    raster_extent <- ext(mean_raster_mask)
    w= abs(raster_extent[2] - raster_extent[1])/10000
    h=abs(raster_extent[4] - raster_extent[3])/10000
    png("outputs/sea_ice.png",width = w*length(chose_list),height = h*length(month_list))
    par(mfrow=c(length(month_list),length(chose_list)))
    count = 1
  }
  for(txt in chose_list){
    if(txt == "Concentration"){
      save_plot(mean_raster_mask, color_breaks$colors, color_breaks$breaks, polygons, year_list[i], month_list[i],subareas_selected)
    }
    if(txt == "Anomalies"){
      download_tiff_1981_2010(paths$tif_mean_path,1981,2010, month_index)
      mean_1981_2010 <- compute_mean_raster_1981_2010(paths$tif_mean_path,month_index)
      mean_1981_2010_mask <- mask_subarea(area_list,mean_1981_2010)
      anomaly_data=compute_anomaly(mean_raster_mask, mean_1981_2010_mask$mean_raster)
      save_anomaly_plot(anomaly_data,mean_raster_mask, year_list[i], month_list[i],subareas_selected)
    }
  }
}
dev.off()



